<img src="EEGRIDAnalytics.png">

In [1]:
import pandas as pd

import ipywidgets as widgets
import json
#from ipyleaflet import Map, GeoJSON

import datetime as dt

import numpy as np
from ipyleaflet import *
from ipywidgets import *



In [2]:
FV_df = pd.read_csv('FV_SW_Outage_Mapping_Stations.csv')
# FV_df1 = FV_df.drop_duplicates(['PSSE New BUS','Latitude','Longitude'])
FV_df1 = FV_df.drop_duplicates(['PSSE New BUS','SSWG Bus Name','Planning Bus Long Name'])

# FV_df1 = FV_df.drop_duplicates()


In [3]:
FV_df1['BusNo&Name'] = FV_df1['PSSE New BUS'].astype('str').str.cat(FV_df1['Planning Bus Long Name'], sep= '-')
Bus_no_list =sorted(FV_df1["BusNo&Name"].unique())
# Bus_no_list

C:\Users\AnnaChang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [4]:
total_day_1 = pd.read_csv('transmission_1015.csv')

genetation_1 = pd.read_csv('busid_generation_1015.csv')

In [5]:
df_1 = pd.DataFrame()
df_2 = pd.DataFrame()

for kk in range(len(total_day_1)):
    aa = FV_df1[FV_df1['PSSE New BUS']== total_day_1['Start Bus ID'].iloc[kk]]
    bb = FV_df1[FV_df1['PSSE New BUS']== total_day_1['End Bus ID'].iloc[kk]]
#     cc = total_day_1[total_day_1['Start Bus ID']== aa['PSSE New BUS'].iloc[0]]
#     dd = total_day_1[total_day_1['Start Bus ID']== bb['PSSE New BUS'].iloc[0]]
#     if aa is not empty
    df_1 = df_1.append(aa)
    df_1 = df_1.append(bb)
    df_1 = df_1.drop_duplicates()
#     df_2 = df_2.append(cc)
#     df_2 = df_2.append(dd)
#     df_2 = df_2.drop_duplicates()
    
# print(df_1)
# print(df_2)  

for ii in range(len(df_1)):
    cc = total_day_1[total_day_1['Start Bus ID']== df_1['PSSE New BUS'].iloc[ii]]
    dd = total_day_1[total_day_1['End Bus ID']== df_1['PSSE New BUS'].iloc[ii]]
#     if cc.empty != True & dd.empty != True:
#         fff.append(df_1[df_1['PSSE New BUS']==df_1['PSSE New BUS'].iloc[ii]]['Latitude'].values)
    
#     if dd.empty != True:
#         ggg.append(df_1[df_1['PSSE New BUS']==df_1['PSSE New BUS'].iloc[ii]]['Longitude'].values)
    
    df_2 = df_2.append(cc)
    df_2 = df_2.append(dd)
    df_2 = df_2.drop_duplicates()
# print(df_2)



In [6]:
df_3 = pd.DataFrame()
df_4 = pd.DataFrame()

for mm in range(len(genetation_1)):
    ee = FV_df1[FV_df1['PSSE New BUS']== genetation_1['bus id'].iloc[mm]]

    df_3 = df_3.append(ee)
    df_3 = df_3.drop_duplicates()

    
# print(df_1)
# print(df_2)  
for aa in range(len(df_3)):
    ff = genetation_1[genetation_1['bus id']== df_3['PSSE New BUS'].iloc[aa]]

    df_4 = df_4.append(ff)
    df_4 = df_4.drop_duplicates()

In [7]:


date_list =sorted(df_2["date"].unique())

date_list.sort(key = lambda date: dt.datetime.strptime(date, '%Y-%m-%d')) 

data_list_last5 = date_list[-5:]

In [8]:
ee_basemaps={}
ee_basemaps['ARCGIS_Satelite'] = TileLayer(url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}')

# Loops through all ipyleaflet basemaps
for item in basemaps.values():
    try:
        name = item['name']
        basemap = 'basemaps.{}'.format(name)
        ee_basemaps[name] = basemap_to_tiles(eval(basemap))
    except:
        for sub_item in item:
            name = item[sub_item]['name']
            basemap = 'basemaps.{}'.format(name)
            basemap = basemap.replace('Mids', 'Modis')
            ee_basemaps[name] = basemap_to_tiles(eval(basemap))
list_maps = list(ee_basemaps.keys())
list_maps_2 = list_maps[0:2]

In [9]:
def change_map_layer(change):
    print('layer change start')
    basemap_name = change['new']
    list_classes1 = [m.layers[jj].__class__.__name__ for jj in range(0,len(m.layers))]
    if len(list_classes1)==1:
        m.add_layer(ee_basemaps[basemap_name])
    else:
        list_classes2 = [m.layers[jj].__class__.__name__ for jj in range(1,len(m.layers))]
        if 'TileLayer' in list_classes2:
            old_basemap1 = m.layers[list_classes2.index('TileLayer')+1]
            m.substitute_layer(old_basemap1, ee_basemaps[basemap_name])
        else:
            m.add_layer(ee_basemaps[basemap_name])
            

In [10]:
def Date_change(change):
    if change.new != None:
        flag = 1
       
        list_classes_BC = [m.layers[kk].__class__.__name__ for kk in range(0,len(m.layers))]
        for items in list_classes_BC:
            if items=='Polyline':
                line_layer_BN = m.layers[list_classes_BC.index(items)]
                m.remove_layer(line_layer_BN)
#         if 'Polyline' in list_classes_BC:
#             line_layer_BN = m.layers[list_classes_BC.index('Polyline')]
#             m.remove_layer(line_layer_BN)
            
#             if 'Polyline' in list_classes_BC:
#                 line_layer_BN = m.layers[list_classes_BC.index('Polyline')]
#                 m.remove_layer(line_layer_BN)
        global df33,df_gene
        df33=df_2[df_2['date']==change.new]
        if df_4.empty != True:
            df_gene = df_4[df_4['date']==change.new]
        else:
            df_gene=pd.DataFrame()
        radio_options.observe(chose_3, names="value")  
        
        if 'radio_value' in globals():       
            if radio_value=='Transmission Outages':
                list_classes_BC = [m.layers[kk].__class__.__name__ for kk in range(0,len(m.layers))]
                for items in list_classes_BC:
                    if items=='Polyline':
                        line_layer_BN = m.layers[list_classes_BC.index(items)]
                        m.remove_layer(line_layer_BN)
        #         df33=df_2[df_2['date']==change.new]
                for mm in range(len(df33)):
                    start_p = df_1[df_1['PSSE New BUS']==df33['Start Bus ID'].iloc[mm]]
        #             y_start = df_1[df_1['PSSE New BUS']==df33['Start Bus ID'].iloc[mm]]

                    end_p = df_1[df_1['PSSE New BUS']==df33['End Bus ID'].iloc[mm]]
        #             y_end = df_1[df_1['PSSE New BUS']==df33['End Bus ID'].iloc[mm]]


                    start_x = start_p['Latitude'].iloc[0]
                    start_y = start_p['Longitude'].iloc[0]
                    start_point=[start_x,start_y]
        #             print(start_point)

        #             end_p = df_1003[df_1003['PSSE New BUS']== day1003_1['End Bus ID'].iloc[kk]]
                    end_x = end_p['Latitude'].iloc[0]
                    end_y = end_p['Longitude'].iloc[0]
                    end_point=[end_x,end_y]
        #             print(end_point)

                    KV_level = start_p['SSWG kV'].iloc[0]
                    if KV_level == '138':
                        color_1 = "blue"
                    elif KV_level == '69':
                        color_1 = "green"
                    elif KV_level == '345':
                        color_1 = "red"

                    line = Polyline(
                            locations=[
                                start_point,
                                end_point
                            ],
                            color=color_1 ,
                            fill=True
                        )


                    message2 = HTML()
                    message2.value ="Outage Line:"+str(df33['Outage Line'].iloc[mm])+'<br>'+'Outage From Bus:'+str(df33['Outage From Bus'].iloc[mm])+'<br>'+\
                        'Start Bus ID:'+str(df33['Start Bus ID'].iloc[mm])+'<br>'+'Outage To Bus:'+str(df33['Outage To Bus'].iloc[mm])+'<br>'+ \
                        'End Bus ID:'+str(df33['End Bus ID'].iloc[mm])

                    line.popup = message2


                    m.add_layer(line)
                    m.center = (start_point)
                    m.zoom = 10
            if radio_value=='Generator Outages':
                list_classes_BC = [m.layers[kk].__class__.__name__ for kk in range(0,len(m.layers))]
                for items in list_classes_BC:
                    if items=='Polyline':
                        line_layer_BN = m.layers[list_classes_BC.index(items)]
                        m.remove_layer(line_layer_BN)

                if df_gene.empty != True:
                    for mm in range(len(df_gene)):
                        start_p = df_3[df_3['PSSE New BUS']==df33['bus id'].iloc[mm]]
            #             y_start = df_1[df_1['PSSE New BUS']==df33['Start Bus ID'].iloc[mm]]

        #                 end_p = df_3[df_3['PSSE New BUS']==df33['End Bus ID'].iloc[mm]]
        #     #             y_end = df_1[df_1['PSSE New BUS']==df33['End Bus ID'].iloc[mm]]


                        start_x = start_p['Latitude'].iloc[0]
                        start_y = start_p['Longitude'].iloc[0]
        #                 start_point=[start_x,start_y]
        #                 print(start_point)
                        marker = Marker(location = (start_x,start_y),draggable=False)
                        marker.icon = AwesomeIcon(name='battery', marker_color='red', icon_color='orange', spin=False)
                        message2 = HTML()
                        message2.value ="Unit Name:"+df_gene['unit name'].iloc[0]+'<br>'+'Unit Size:'+df_gene['unit size'].iloc[0]+'<br>'+ \
                                        'Fuel Type:'+df_gene['fuel type'].iloc[0]+'<br>'+'outage start:'+df_gene['outage start'].iloc[0]+ \
                                        '<br>'+'outage end:'+df_gene['outage end'].iloc[0]
                        marker.popup = message2
                        m.add_layer(marker)
                        m.center = (start_x,start_y)
                        m.zoom = 18




            if radio_value=='Show Both':
                list_classes_BC = [m.layers[kk].__class__.__name__ for kk in range(0,len(m.layers))]
                for items in list_classes_BC:
                    if items=='Polyline':
                        line_layer_BN = m.layers[list_classes_BC.index(items)]
                        m.remove_layer(line_layer_BN)

                for mm in range(len(df33)):
                    start_p = df_1[df_1['PSSE New BUS']==df33['Start Bus ID'].iloc[mm]]
        #             y_start = df_1[df_1['PSSE New BUS']==df33['Start Bus ID'].iloc[mm]]

                    end_p = df_1[df_1['PSSE New BUS']==df33['End Bus ID'].iloc[mm]]
        #             y_end = df_1[df_1['PSSE New BUS']==df33['End Bus ID'].iloc[mm]]


                    start_x = start_p['Latitude'].iloc[0]
                    start_y = start_p['Longitude'].iloc[0]
                    start_point=[start_x,start_y]
        #             print(start_point)

        #             end_p = df_1003[df_1003['PSSE New BUS']== day1003_1['End Bus ID'].iloc[kk]]
                    end_x = end_p['Latitude'].iloc[0]
                    end_y = end_p['Longitude'].iloc[0]
                    end_point=[end_x,end_y]
        #             print(end_point)

                    KV_level = start_p['SSWG kV'].iloc[0]
                    if KV_level == '138':
                        color_1 = "blue"
                    elif KV_level == '69':
                        color_1 = "green"
                    elif KV_level == '345':
                        color_1 = "red"





                    line = Polyline(
                            locations=[
                                start_point,
                                end_point
                            ],
                            color=color_1 ,
                            fill=True
                        )


                    message2 = HTML()
                    message2.value ="Outage Line:"+str(df33['Outage Line'].iloc[mm])+'<br>'+'Outage From Bus:'+str(df33['Outage From Bus'].iloc[mm])+'<br>'+\
                        'Start Bus ID:'+str(df33['Start Bus ID'].iloc[mm])+'<br>'+'Outage To Bus:'+str(df33['Outage To Bus'].iloc[mm])+'<br>'+ \
                        'End Bus ID:'+str(df33['End Bus ID'].iloc[mm])

                    line.popup = message2


                    m.add_layer(line)
                    m.center = (start_point)
        #             m.zoom = 10

                if df_gene.empty != True:
                    for mm in range(len(df_gene)):
                        start_p = df_3[df_3['PSSE New BUS']==df33['bus id'].iloc[mm]]
            #             y_start = df_1[df_1['PSSE New BUS']==df33['Start Bus ID'].iloc[mm]]

        #                 end_p = df_3[df_3['PSSE New BUS']==df33['End Bus ID'].iloc[mm]]
        #     #             y_end = df_1[df_1['PSSE New BUS']==df33['End Bus ID'].iloc[mm]]


                        start_x = start_p['Latitude'].iloc[0]
                        start_y = start_p['Longitude'].iloc[0]
        #                 start_point=[start_x,start_y]
        #                 print(start_point)
                        marker = Marker(location = (start_x,start_y),draggable=False)
                        marker.icon = AwesomeIcon(name='battery', marker_color='red', icon_color='orange', spin=False)
                        message2 = HTML()
                        message2.value ="Unit Name:"+df_gene['unit name'].iloc[0]+'<br>'+'Unit Size:'+df_gene['unit size'].iloc[0]+'<br>'+ \
                                        'Fuel Type:'+df_gene['fuel type'].iloc[0]+'<br>'+'outage start:'+df_gene['outage start'].iloc[0]+ \
                                        '<br>'+'outage end:'+df_gene['outage end'].iloc[0]
                        marker.popup = message2
                        m.add_layer(marker)
                        m.center = (start_x,start_y)
                m.zoom = 10


In [11]:
def Bus_No_control(change):
    if change.new != None:
        flag = 1
        df88_1=FV_df1[FV_df1['BusNo&Name'] == change.new]
        x = df88_1['Latitude']
        y = df88_1['Longitude']
        bus_fp = df88_1['PSSE New BUS'].iloc[0] 
        marker = Marker(location = (x.iloc[0],y.iloc[0]),draggable=False)
        if (bus_fp == 38069) or (bus_fp == 38001):
            marker.icon = AwesomeIcon(name='battery', marker_color='green', icon_color='darkgreen', spin=False)
        
        message2 = HTML()
        message2.value ="Bus Name:"+df88_1['Planning Bus Long Name'].iloc[0]+'<br>'+'County:'+df88_1['County'].iloc[0] \
                        +'<br>'+'KV:'+df88_1['SSWG kV'].iloc[0]+'<br>'+'Settlement Zone:'+df88_1['Settlement Zone'].iloc[0]\
                        +'<br>'+'Weather Zone:'+df88_1['Weather Zone'].iloc[0]
        marker.popup = message2
        
        

        m.add_layer(marker)
        m.center = (x.iloc[0],y.iloc[0])
        m.zoom = 18


In [12]:
def chose_3(button):
    global radio_value
    radio_value = button['new']
    if button['new']=='Transmission Outages':
        list_classes_BC = [m.layers[kk].__class__.__name__ for kk in range(0,len(m.layers))]
        for items in list_classes_BC:
            if items=='Polyline':
                line_layer_BN = m.layers[list_classes_BC.index(items)]
                m.remove_layer(line_layer_BN)
#         df33=df_2[df_2['date']==change.new]
        for mm in range(len(df33)):
            start_p = df_1[df_1['PSSE New BUS']==df33['Start Bus ID'].iloc[mm]]
#             y_start = df_1[df_1['PSSE New BUS']==df33['Start Bus ID'].iloc[mm]]
            
            end_p = df_1[df_1['PSSE New BUS']==df33['End Bus ID'].iloc[mm]]
#             y_end = df_1[df_1['PSSE New BUS']==df33['End Bus ID'].iloc[mm]]
            
            
            start_x = start_p['Latitude'].iloc[0]
            start_y = start_p['Longitude'].iloc[0]
            start_point=[start_x,start_y]
#             print(start_point)

#             end_p = df_1003[df_1003['PSSE New BUS']== day1003_1['End Bus ID'].iloc[kk]]
            end_x = end_p['Latitude'].iloc[0]
            end_y = end_p['Longitude'].iloc[0]
            end_point=[end_x,end_y]
#             print(end_point)
            
            KV_level = start_p['SSWG kV'].iloc[0]
            if KV_level == '138':
                color_1 = "blue"
            elif KV_level == '69':
                color_1 = "green"
            elif KV_level == '345':
                color_1 = "red"
   
            line = Polyline(
                    locations=[
                        start_point,
                        end_point
                    ],
                    color=color_1 ,
                    fill=True
                )
            
            
            message2 = HTML()
            message2.value ="Outage Line:"+str(df33['Outage Line'].iloc[mm])+'<br>'+'Outage From Bus:'+str(df33['Outage From Bus'].iloc[mm])+'<br>'+\
                'Start Bus ID:'+str(df33['Start Bus ID'].iloc[mm])+'<br>'+'Outage To Bus:'+str(df33['Outage To Bus'].iloc[mm])+'<br>'+ \
                'End Bus ID:'+str(df33['End Bus ID'].iloc[mm])

            line.popup = message2


            m.add_layer(line)
            m.center = (start_point)
            m.zoom = 10
    if button['new']=='Generator Outages':
        list_classes_BC = [m.layers[kk].__class__.__name__ for kk in range(0,len(m.layers))]
        for items in list_classes_BC:
            if items=='Polyline':
                line_layer_BN = m.layers[list_classes_BC.index(items)]
                m.remove_layer(line_layer_BN)
                
        if df_gene.empty != True:
            for mm in range(len(df_gene)):
                start_p = df_3[df_3['PSSE New BUS']==df33['bus id'].iloc[mm]]
    #             y_start = df_1[df_1['PSSE New BUS']==df33['Start Bus ID'].iloc[mm]]

#                 end_p = df_3[df_3['PSSE New BUS']==df33['End Bus ID'].iloc[mm]]
#     #             y_end = df_1[df_1['PSSE New BUS']==df33['End Bus ID'].iloc[mm]]


                start_x = start_p['Latitude'].iloc[0]
                start_y = start_p['Longitude'].iloc[0]
#                 start_point=[start_x,start_y]
#                 print(start_point)
                marker = Marker(location = (start_x,start_y),draggable=False)
                marker.icon = AwesomeIcon(name='battery', marker_color='red', icon_color='orange', spin=False)
                message2 = HTML()
                message2.value ="Unit Name:"+df_gene['unit name'].iloc[0]+'<br>'+'Unit Size:'+df_gene['unit size'].iloc[0]+'<br>'+ \
                                'Fuel Type:'+df_gene['fuel type'].iloc[0]+'<br>'+'outage start:'+df_gene['outage start'].iloc[0]+ \
                                '<br>'+'outage end:'+df_gene['outage end'].iloc[0]
                marker.popup = message2
                m.add_layer(marker)
                m.center = (start_x,start_y)
                m.zoom = 18

                
                
                
    if button['new']=='Show Both':
        list_classes_BC = [m.layers[kk].__class__.__name__ for kk in range(0,len(m.layers))]
        for items in list_classes_BC:
            if items=='Polyline':
                line_layer_BN = m.layers[list_classes_BC.index(items)]
                m.remove_layer(line_layer_BN)
                
        for mm in range(len(df33)):
            start_p = df_1[df_1['PSSE New BUS']==df33['Start Bus ID'].iloc[mm]]
#             y_start = df_1[df_1['PSSE New BUS']==df33['Start Bus ID'].iloc[mm]]
            
            end_p = df_1[df_1['PSSE New BUS']==df33['End Bus ID'].iloc[mm]]
#             y_end = df_1[df_1['PSSE New BUS']==df33['End Bus ID'].iloc[mm]]
            
            
            start_x = start_p['Latitude'].iloc[0]
            start_y = start_p['Longitude'].iloc[0]
            start_point=[start_x,start_y]
#             print(start_point)

#             end_p = df_1003[df_1003['PSSE New BUS']== day1003_1['End Bus ID'].iloc[kk]]
            end_x = end_p['Latitude'].iloc[0]
            end_y = end_p['Longitude'].iloc[0]
            end_point=[end_x,end_y]
#             print(end_point)
            
            KV_level = start_p['SSWG kV'].iloc[0]
            if KV_level == '138':
                color_1 = "blue"
            elif KV_level == '69':
                color_1 = "green"
            elif KV_level == '345':
                color_1 = "red"
            
            
            
            

            line = Polyline(
                    locations=[
                        start_point,
                        end_point
                    ],
                    color=color_1 ,
                    fill=True
                )
            
            
            message2 = HTML()
            message2.value ="Outage Line:"+str(df33['Outage Line'].iloc[mm])+'<br>'+'Outage From Bus:'+str(df33['Outage From Bus'].iloc[mm])+'<br>'+\
                'Start Bus ID:'+str(df33['Start Bus ID'].iloc[mm])+'<br>'+'Outage To Bus:'+str(df33['Outage To Bus'].iloc[mm])+'<br>'+ \
                'End Bus ID:'+str(df33['End Bus ID'].iloc[mm])

            line.popup = message2


            m.add_layer(line)
            m.center = (start_point)
#             m.zoom = 10
            
        if df_gene.empty != True:
            for mm in range(len(df_gene)):
                start_p = df_3[df_3['PSSE New BUS']==df33['bus id'].iloc[mm]]
    #             y_start = df_1[df_1['PSSE New BUS']==df33['Start Bus ID'].iloc[mm]]

#                 end_p = df_3[df_3['PSSE New BUS']==df33['End Bus ID'].iloc[mm]]
#     #             y_end = df_1[df_1['PSSE New BUS']==df33['End Bus ID'].iloc[mm]]


                start_x = start_p['Latitude'].iloc[0]
                start_y = start_p['Longitude'].iloc[0]
#                 start_point=[start_x,start_y]
#                 print(start_point)
                marker = Marker(location = (start_x,start_y),draggable=False)
                marker.icon = AwesomeIcon(name='battery', marker_color='red', icon_color='orange', spin=False)
                message2 = HTML()
                message2.value ="Unit Name:"+df_gene['unit name'].iloc[0]+'<br>'+'Unit Size:'+df_gene['unit size'].iloc[0]+'<br>'+ \
                                'Fuel Type:'+df_gene['fuel type'].iloc[0]+'<br>'+'outage start:'+df_gene['outage start'].iloc[0]+ \
                                '<br>'+'outage end:'+df_gene['outage end'].iloc[0]
                marker.popup = message2
                m.add_layer(marker)
                m.center = (start_x,start_y)
        m.zoom = 10
        

In [13]:


center=(31.432,-102.4377)
#print(center)
m = Map(center=center, zoom_control=False, zoom=5,inertia = True, scroll_wheel_zoom = True, inertia_max_speed = 1500, preferCanvas = True)
m.layout.height='800px'

full_screen = FullScreenControl(position='bottomleft')
m.add_control(full_screen)
m.add_control(ZoomControl(position='bottomleft'))
m

Map(center=[31.432, -102.4377], controls=(AttributionControl(options=['position', 'prefix'], position='bottomr…

IndexError: single positional indexer is out-of-bounds

In [14]:
dd1 = widgets.Dropdown(options=data_list_last5,value=None,description='Date:',layout=Layout(width='250px'))
dd1.observe(Date_change, names="value")
widget_control2 = WidgetControl(widget=dd1, position='topleft')
m.add_control(widget_control2)

radio_options = RadioButtons(options=['Transmission Outages', 'Generator Outages', 'Show Both'],disabled=False, value = None)
radio_options.observe(chose_3, names="value")
widget_control4 = WidgetControl(widget=radio_options, position='topleft')
m.add_control(widget_control4)

Map_layer = widgets.Dropdown(options=list_maps_2,value=None,description='BaseMap:')
Map_layer.observe(change_map_layer, 'value')
base_layer = WidgetControl(widget=Map_layer, position='topright')
m.add_control(base_layer)


bus_number = widgets.Dropdown(options=Bus_no_list,value=None,description='Bus NO.:',layout=Layout(width='250px'))
bus_number.observe(Bus_No_control, names="value")
widget_control3 = WidgetControl(widget=bus_number, position='topright')
m.add_control(widget_control3)

In [15]:
with open('2343.geojson', 'r') as f:
    data = json.load(f)
features = data['features']
for i in range(len(features)):
    location=(features[i]['geometry']['coordinates'][1],features[i]['geometry']['coordinates'][0])
    PSSE_New_BUS = features[i]['properties']['PSSE New BUS']
#     print(PSSE_New_BUS)
#     PSSE_New_BUS = str(PSSE_New_BUS)
    SSWG_Bus_Name = features[i]['properties']['SSWG Bus Name']
    SSWG_kV = features[i]['properties']['SSWG kV']
    County = features[i]['properties']['County']
#     County = str(County)
    Weather_Zone = features[i]['properties']['Weather Zone']
    
    Settlement_Zone = features[i]['properties']['Settlement Zone']
#     Settlement_Zone = str(Settlement_Zone)
#     SSWG_Bus_Name = features[i]['properties']['SSWG_Bus_Name']
#     SSWG_Bus_Name = features[i]['properties']['SSWG_Bus_Name']
    marker = Marker(location=location,draggable=False)
    message2 = HTML()
    message2.value ="Bus No.:"+str(PSSE_New_BUS)+'<br>'+"Bus Name:"+str(SSWG_Bus_Name)+'<br>'+'County:'+str(County) \
                    +'<br>'+'KV:'+str(SSWG_kV)+'<br>'+'Settlement Zone:'+str(Settlement_Zone)\
                    +'<br>'+'Weather Zone:'+str(Weather_Zone)
#     marker.popup = message2
    if (PSSE_New_BUS== '38069') or (PSSE_New_BUS == '38001'):
        marker.icon = AwesomeIcon(name='battery', marker_color='green', icon_color='darkgreen', spin=False)



#     html = """
#     <p>
#       <h4><b>PSSE_New_BUS</b>:        """ + str(PSSE_New_BUS)+ """</h4>
#       <h4><b>SSWG_Bus_Name</b>:        """ + str(SSWG_Bus_Name)+ """</h4>
#       <h4><b>SSWG_kV</b>:        """ + str(SSWG_kV)+ """</h4>
#       <h4><b>County</b>:        """ + str(County)+ """</h4>
#       <h4><b>Weather_Zone</b>:        """ + str(Weather_Zone)+ """</h4>
#       <h4><b>Settlement_Zone</b>:        """ + str(Settlement_Zone)+ """</h4>
#     </p>
#     """
    

    # Popup associated to a layer
    marker.popup = message2
    m.add_layer(marker)